<a href="https://colab.research.google.com/github/tsitsvero/rotor-gp/blob/main/code/descriptors_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
from rascaline import SoapPowerSpectrum

# https://github.com/Luthaf/rascaline/blob/0e3e5e9dadbc8595d3adce4f3ff01a98e456c56f/python/rascaline/examples/first-calculation.py#L259

HYPER_PARAMETERS = {
    "cutoff": 4.0,
    "max_radial": 4,
    "max_angular": 4,
    "atomic_gaussian_width": 0.3,
    "center_atom_weight": 1.0,
    "radial_basis": {
        "Gto": {},
    },
    "cutoff_function": {
        "ShiftedCosine": {"width": 0.5},
    },
    # "normalize": True,
}

calculator = SoapPowerSpectrum(**HYPER_PARAMETERS)

In [40]:
from ase import io
import os
from tqdm import tqdm

# frames = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="22")
frames = io.read(os.path.expanduser("/home/mm/repos/data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj"), index="22:23")

for f in frames:
        f.wrap(eps=1e-8)

In [49]:
%%time

descriptor = calculator.compute(tqdm(frames), gradients=["positions"])
print("before: ", len(descriptor.keys))


descriptor = descriptor.keys_to_samples("species_center")
print("after: ", len(descriptor.keys))
descriptor = descriptor.keys_to_properties(["species_neighbor_1", "species_neighbor_2"])
print("after: ", len(descriptor.keys))

100%|██████████| 1/1 [00:00<00:00, 1671.70it/s]


before:  55
after:  15
after:  1
CPU times: user 1.13 s, sys: 1.28 s, total: 2.41 s
Wall time: 766 ms


In [ ]:
# positions_gradient = block.gradient("positions")


In [48]:
# descriptor.block(0).values.shape
descriptor.block(0).gradient("positions").values.shape

(14540, 3, 1200)

In [174]:
descriptor_gradients = calculator.compute([frames], gradients=["positions"])
block_gradients = descriptor_gradients.block(0)
gradient_position = block_gradients.gradient("positions")


AttributeError: 'Gradient' object has no attribute 'values'

In [178]:
gradient_position

Gradient TensorBlock
parameter: 'positions'
samples (5144): ['sample', 'structure', 'atom']
components (3): ['direction']
properties (80): ['l', 'n1', 'n2']

In [172]:
gradient_position.values

AttributeError: 'Gradient' object has no attribute 'values'

In [162]:
gradient_position

Gradient TensorBlock
parameter: 'positions'
samples (5144): ['sample', 'structure', 'atom']
components (3): ['direction']
properties (80): ['l', 'n1', 'n2']

In [150]:
descriptor_gradients.block(1).values.shape

(300, 80)

In [100]:
descriptor.block().values[1]

ExternalCpuArray([ 1.47592977e-03, -4.55155770e-03,  1.70581386e-02,
                   1.29862627e-02, -4.55155770e-03,  1.40363572e-02,
                  -5.26048759e-02, -4.00477890e-02,  1.70581386e-02,
                  -5.26048759e-02,  1.97150366e-01,  1.50089438e-01,
                   1.29862627e-02, -4.00477890e-02,  1.50089438e-01,
                   1.14262224e-01,  4.61459301e-04, -1.48872261e-03,
                   4.91209845e-03,  4.54660033e-03, -1.48872261e-03,
                   4.80284313e-03, -1.58467561e-02, -1.46679500e-02,
                   4.91209845e-03, -1.58467561e-02,  5.22892654e-02,
                   4.83967757e-02,  4.54660033e-03, -1.46679500e-02,
                   4.83967757e-02,  4.47966005e-02,  1.41418027e-04,
                  -3.84921709e-04,  1.95688590e-03,  8.30447421e-04,
                  -3.84921709e-04,  1.04776866e-03, -5.32605594e-03,
                  -2.26039374e-03,  1.95688590e-03, -5.32605594e-03,
                   2.70804562e-02,

In [ ]:
import numpy as np
from matplotlib import pyplot as plt


for i in range(684):
        plt.plot(descriptor.block().values[i])

plt.show()
# np.sum( descriptor.block().values[0]**2 )

for i in range(684):
        plt.plot(soap_array[i])

plt.show()

In [ ]:
descriptor.block().values[0]

# With rascal

In [3]:
from rascal.representations import SphericalInvariants

soap_params = dict(soap_type="PowerSpectrum",
        interaction_cutoff=4.0,
        max_radial=4,
        max_angular=4,
        gaussian_sigma_constant=0.3,
        gaussian_sigma_type="Constant",
        cutoff_function_type="RadialScaling",
        cutoff_smooth_width=0.1, # 0.1 is way better than 0.5
        cutoff_function_parameters=
                dict(
                        rate=1,
                        scale=3.5,
                        exponent=4
                        ),
        radial_basis="GTO",
        normalize=True, # setting False makes model untrainable
        #   optimization=
        #         dict(
        #                 Spline=dict(
        #                    accuracy=1.0e-05
        #                 )
        #             ),
        compute_gradients=True, # for energies gradients are ignored
        expansion_by_species_method='structure wise'
        )

hypers =  soap_params#HYPER_PARAMETERS

soap = SphericalInvariants(**hypers)

In [6]:
%%time
managers = soap.transform(frames)
soap_array = managers.get_features(soap)

soap_grad_array = managers.get_features_gradient(soap)

CPU times: user 1.1 s, sys: 324 ms, total: 1.42 s
Wall time: 1.42 s


In [8]:
import numpy as np
np.sum( soap_array[40]**2 )

1.0000000000000002

In [30]:
# https://lab-cosmo.github.io/metatensor/latest/reference/python/block.html#metatensor.TensorBlock

import numpy as np
from metatensor import Labels, TensorBlock
block = TensorBlock(
    values=np.full((3, 1, 5), 1.0),
    samples=Labels(["structure"], np.array([[0], [2], [4]])),
    components=[Labels.range("component", 1)],
    properties=Labels.range("property", 5),
)
positions_gradient = TensorBlock(
    values=np.full((2, 3, 1, 5), 11.0),
    samples=Labels(["sample", "atom"], np.array([[0, 2], [2, 3]])),
    components=[
        Labels.range("direction", 3),
        Labels.range("component", 1),
    ],
    properties=Labels.range("property", 5),
)

In [12]:
import os

os.chdir("test")

In [13]:
os.getcwd()

'/home/mm/repos/rotor-gp/code/test'

In [7]:
import cclib

filename = "xtb.out"
data = cclib.io.ccread(filename)
# print("There are %i atoms and %i MOs" % (data.natom, data.nmo))

ModuleNotFoundError: No module named 'cclib'

In [8]:
from ase.build import molecule

from xtb_ase import XTB, XTBProfile

atoms = molecule("H2O")
profile = XTBProfile(["xtb", "--gfn2"])
# profile = XTBProfile(["/home/mm/repos/xtb/build/xtb", "--gfnff"])
atoms.calc = XTB(profile=profile)

atoms.get_potential_energy()

# print(atoms.calc.results)

normal termination of xtb
Unable to determine the type of logfile [PosixPath('xtb.out'), PosixPath('gradient')], try the fallback mechanism


AttributeError: 'NoneType' object has no attribute 'scfenergies'

In [10]:
import ase
ase.__version__

'3.23.0b1'

In [40]:
XTB.output_file


AttributeError: type object 'XTB' has no attribute 'output_file'

In [4]:
import cclib

In [22]:
from ase.build import bulk
from xtb_ase import XTB, XTBProfile

atoms = bulk("Cu")
profile = XTBProfile(["/home/mm/repos/xtb/build/xtb", "--gfnff"])
atoms.calc = XTB(profile=profile)

atoms.get_potential_energy()

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Unable to determine the type of logfile [PosixPath('xtb.out'), PosixPath('gradient')], try the fallback mechanism


AttributeError: 'NoneType' object has no attribute 'scfenergies'